In [1]:
import os
import torch
from torch.utils import data
from PIL import Image
import numpy as np
from torchvision import transforms
import matplotlib.pyplot as plt

#对图片进行等比例裁剪
imgs_root=os.listdir('./SCUT-FBP5500_v2/test')
imgs=[]
for k in imgs_root:
    im = Image.open(os.path.join('./SCUT-FBP5500_v2/test',k))
    w, h = im.size
    if w!=350 or h!=350:
        if w >= h:
            im1 = im.resize((350*w//h,350),Image.ANTIALIAS)
        else:
            im1 = im.resize((350,350*h//w),Image.ANTIALIAS)
        im2 = im1.crop((0,0,350,350))
        im2.save(os.path.join('./SCUT-FBP5500_v2/test',k))

# 导入图片        
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 标准化至[-1,1]
])
def imshow(img):
    img=img/2+0.5  #反归一化操作，以便原样显示图片
    npimg=img.numpy()
    plt.imshow(np.transpose(npimg,(1,2,0)))
    plt.show()


#定义自己的数据集合
class FlameSet(data.Dataset):
    def __init__(self,root):
        # 所有图片的绝对路径
        imgs_root=os.listdir(root)
        imgs=[]
        for k in imgs_root:
            imgs.append((os.path.join(root,k),4.5))
        self.imgs=imgs
        self.transforms=transform

    def __getitem__(self, index):
        img_path,label = self.imgs[index]
        pil_img = Image.open(img_path)
        if self.transforms:
            data = self.transforms(pil_img)
        else:
            pil_img = np.asarray(pil_img)
            data = torch.from_numpy(pil_img)
        return data,label

    def __len__(self):
        return len(self.imgs)

testset=FlameSet('./SCUT-FBP5500_v2/test')

from torch.utils.data import DataLoader

testloader=DataLoader(testset,batch_size=4,shuffle=False,num_workers=0)

#定义卷积神经网络的网络模型
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 36, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.aap=nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(36, 1)
        
    def forward(self,x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x=self.aap(x)
        x=x.view(x.shape[0],-1)
        x=self.fc1(x)
        x = x.squeeze(-1)
        return x

# 载入训练好的模型
net=Net()
net.load_state_dict(torch.load('./FBP/net2_2.pth'))

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)

outputs

tensor([2.6641, 2.8846])